In [10]:

from termcolor import colored
from IPython.display import Markdown, display

from langchain_pinecone import PineconeVectorStore
from langchain_core.prompts import PromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough, RunnableParallel
from langchain_openai import ChatOpenAI, OpenAIEmbeddings

import os
from dotenv import load_dotenv
load_dotenv()

/Users/joaovictorfarrulladarze/.pyenv/versions/3.10.3/lib/python3.10/site-packages/pinecone/data/index.py:1: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from tqdm.autonotebook import tqdm


True

In [7]:
openai_key = os.getenv("OPENAI_API_KEY")
pinecone_key = os.getenv("PINECONE_API_KEY")

In [12]:
Prompt_template = PromptTemplate.from_template(
  """
  Você é um assistente de notícias esportivas.
  Seu trabalho é responder perguntas sobre notícias que acontecem no mundo do esporte.
  Você SEMPRE deve responder com base nas informações recebidas.
  Se não souber a resposta, apenas diga que não sabe.
  SEMPRE responda na língua em que a pergunta foi feita. 

  Informações:
  {context}

  Pergunta: {question}

  Resposta:
  """
)

In [14]:
# RAG Chain
embeddings = OpenAIEmbeddings()

index_name = "news-index"
vectorstore = PineconeVectorStore(index_name, embedding=embeddings)
# Retrieve and generate using the relevant snippets of the blog.
retriever = vectorstore.as_retriever(search_kwargs={"k": 10})


llm = ChatOpenAI(model_name="gpt-3.5-turbo", temperature=0)


def format_docs(docs):
    return "\n\n".join(doc.page_content for doc in docs)


rag_chain_from_docs = (
    RunnablePassthrough.assign(context=(lambda x: format_docs(x["context"])))
    | Prompt_template
    | llm
    | StrOutputParser()
)

rag_chain_with_source = RunnableParallel(
    {"context": retriever, "question": RunnablePassthrough()}
).assign(answer=rag_chain_from_docs)

In [ ]:
question = "Qual o time de futebol mais antigo do Brasil?"
res = rag_chain_with_source.invoke(question)